# Riesgo por incumplimiento de pagos en créditos de vivienda
---------------------------------------------------------------------------------------------------------------------------

**Autor:** Cristhian David Tafur Hernández

**Asesor:** Raul Ramos


Notebook: 04 Preprocesamiento del dataset para ampliar la información de entrenamiento

Objetivo de la código:

Preprocesar dos de las tablas complementarias a la data principal y unirlas a la misma con el fin de ampliar la información disponible para la ejecución de los algoritmos de clasificación.


## Preparación de entorno de trabajo

In [ ]:
# Básicas
# ==============================================================================
import pandas as pd
import numpy as np

# Preporcesamiento de datos
# ==============================================================================
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_lectura ='/content/drive/MyDrive/Monografia/data_modelo/'

# Bureau

In [ ]:
#Carga de datos
d = pd.read_csv (ruta_lectura + 'bureau.csv')

In [ ]:
# Dimensionalidad de "Application"
print('Dimensionalidad de los datos:')
print('-----------------------------------------------')
print('Número de muestras: ', d.shape[0], 'Créditos/prestamos')
print('Número de features en application: ', d.shape[1]-2) #menos 2 porque se excluye el Target y el "id" de la muestra

Dimensionalidad de los datos:
-----------------------------------------------
Número de muestras:  1716428 Créditos/prestamos
Número de features en application:  15


In [ ]:
#Tipos de variables.
det_var = pd.read_excel(ruta_lectura + 'type_variables.xlsx')
det_app = det_var[det_var['Table']=='bureau.csv']

pd.DataFrame(det_app.groupby('type').count()['nun_var']).sort_values(by = 'nun_var')

,nun_var
type,
ID,2
C,3
N,12


In [ ]:
d_count = pd.DataFrame(d.groupby('SK_ID_CURR').size())
d_count.describe()

,0
count,305811.000000
mean,5.612709
std,4.430354
min,1.000000
25%,2.000000
50%,4.000000
75%,8.000000
max,116.000000


### Features Númericas

In [ ]:
#Identificación de variables numéricas
det_app_N = det_app[(det_app['type']=='N')]
var_n = det_app_N['column'].tolist()
var_n.append('SK_ID_CURR')

dn = d[var_n]

In [ ]:
# Estadísticas descriptivas de las variables 
dn_describe=dn.describe().T
pd.set_option('display.max_rows', dn_describe.shape[0]+1)
dn_describe

,count,mean,std,min,25%,50%,75%,max
DAYS_CREDIT,1716428.0,-1142.107685,7.951649e+02,-2922.000,-1666.00,-987.0,-474.0,0.000000e+00
CREDIT_DAY_OVERDUE,1716428.0,0.818167,3.654443e+01,0.000,0.00,0.0,0.0,2.792000e+03
DAYS_CREDIT_ENDDATE,1610875.0,510.517362,4.994220e+03,-42060.000,-1138.00,-330.0,474.0,3.119900e+04
DAYS_ENDDATE_FACT,1082775.0,-1017.437148,7.140106e+02,-42023.000,-1489.00,-897.0,-425.0,0.000000e+00
AMT_CREDIT_MAX_OVERDUE,591940.0,3825.417661,2.060316e+05,0.000,0.00,0.0,0.0,1.159872e+08
CNT_CREDIT_PROLONG,1716428.0,0.006410,9.622391e-02,0.000,0.00,0.0,0.0,9.000000e+00
AMT_CREDIT_SUM,1716415.0,354994.591918,1.149811e+06,0.000,51300.00,125518.5,315000.0,5.850000e+08
AMT_CREDIT_SUM_DEBT,1458759.0,137085.119952,6.774011e+05,-4705600.320,0.00,0.0,40153.5,1.701000e+08
AMT_CREDIT_SUM_LIMIT,1124648.0,6229.514980,4.503203e+04,-586406.115,0.00,0.0,0.0,4.705600e+06
AMT_CREDIT_SUM_OVERDUE,1716428.0,37.912758,5.937650e+03,0.000,0.00,0.0,0.0,3.756681e+06


In [ ]:
dn_e = dn.groupby('SK_ID_CURR').median()
dn_e.fillna(dn_e.median(), inplace =True)

### Features Categoricas

In [ ]:
#Identificación de variables numéricas
det_app_C = det_app[(det_app['type']=='C')]
var_c = det_app_C['column'].tolist()
var_c.append('SK_ID_CURR')

dc = d[var_c]

In [ ]:
print('Visualización de las clases de las variables categóricas')
print('---------------------------------------------')
for c in var_c[:-1]:
    print ("%10s"%c, np.unique(dc[c].dropna()))

Visualización de las clases de las variables categóricas
---------------------------------------------
CREDIT_ACTIVE ['Active' 'Bad debt' 'Closed' 'Sold']
CREDIT_CURRENCY ['currency 1' 'currency 2' 'currency 3' 'currency 4']
CREDIT_TYPE ['Another type of loan' 'Car loan' 'Cash loan (non-earmarked)'
 'Consumer credit' 'Credit card' 'Interbank credit'
 'Loan for business development'
 'Loan for purchase of shares (margin lending)'
 'Loan for the purchase of equipment'
 'Loan for working capital replenishment' 'Microloan'
 'Mobile operator loan' 'Mortgage' 'Real estate loan'
 'Unknown type of loan']


In [ ]:
dc.set_index('SK_ID_CURR', inplace=True)

In [ ]:
col_str = (dc.dtypes == object)
col_str = [c for c in col_str.index if col_str[c]]

col_du = (dc.dtypes != object)
col_du = [c for c in col_du.index if col_du[c]]

In [ ]:
dc_str = pd.get_dummies(dc[col_str], prefix=col_str)

In [ ]:
dc_str.reset_index(inplace=True)

In [ ]:
dc_str = dc_str.groupby('SK_ID_CURR').max().reset_index()

## Unión de datos Bureau procesados

In [ ]:
# Validación de que las variables transformadas no contengan valores faltantes
dc_str.isnull().sum().sum(), dn_e.isnull().sum().sum()

(0, 0)

In [ ]:
bureau = pd.merge(dn_e,dc_str, how ='outer', on = 'SK_ID_CURR') 

In [ ]:
bureau.drop_duplicates(subset=['SK_ID_CURR'], inplace= True)

In [ ]:
col_pre = bureau.columns.tolist()[1:]
for i in col_pre:
  bureau.rename(columns={i:i+'_bur'}, inplace = True)

In [ ]:
# Validación de que las variables transformadas no contengan valores faltantes
bureau.isnull().sum().sum()

0

In [ ]:
bureau.to_csv(ruta_lectura + "bureau_procesada.csv")

# Previous

In [ ]:
#Carga de datos
d = pd.read_csv (ruta_lectura + 'previous_application.csv')

In [ ]:
# Dimensionalidad de "Application"
print('Dimensionalidad de los datos:')
print('-----------------------------------------------')
print('Número de muestras: ', d.shape[0], 'Créditos/prestamos')
print('Número de features en application: ', d.shape[1]-2) #menos 2 porque se excluye el Target y el "id" de la muestra

Dimensionalidad de los datos:
-----------------------------------------------
Número de muestras:  1670214 Créditos/prestamos
Número de features en application:  35


In [ ]:
#Tipos de variables.
det_var = pd.read_excel(ruta_lectura + 'type_variables.xlsx')
det_app = det_var[det_var['Table']=='previous_application.csv']

pd.DataFrame(det_app.groupby('type').count()['nun_var']).sort_values(by = 'nun_var')

,nun_var
type,
NN,1
ID,2
N,17
C,18


In [ ]:
d_count = pd.DataFrame(d.groupby('SK_ID_CURR').size())
d_count.describe()

,0
count,338857.000000
mean,4.928964
std,4.220716
min,1.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,77.000000


### Features Númericas

In [ ]:
#Identificación de variables numéricas
det_app_N = det_app[(det_app['type']=='N')]
var_n = det_app_N['column'].tolist()
var_n.append('SK_ID_CURR')

dn = d[var_n]

In [ ]:
# Estadísticas descriptivas de las variables 
dn_describe=dn.describe().T
pd.set_option('display.max_rows', dn_describe.shape[0]+1)
dn_describe

,count,mean,std,min,25%,50%,75%,max
AMT_ANNUITY,1297979.0,15955.120659,14782.137335,0.000000,6321.780000,11250.000000,20658.420000,418058.145
AMT_APPLICATION,1670214.0,175233.860360,292779.762387,0.000000,18720.000000,71046.000000,180360.000000,6905160.000
AMT_CREDIT,1670213.0,196114.021218,318574.616546,0.000000,24160.500000,80541.000000,216418.500000,6905160.000
AMT_DOWN_PAYMENT,774370.0,6697.402139,20921.495410,-0.900000,0.000000,1638.000000,7740.000000,3060045.000
AMT_GOODS_PRICE,1284699.0,227847.279283,315396.557937,0.000000,50841.000000,112320.000000,234000.000000,6905160.000
HOUR_APPR_PROCESS_START,1670214.0,12.484182,3.334028,0.000000,10.000000,12.000000,15.000000,23.000
RATE_DOWN_PAYMENT,774370.0,0.079637,0.107823,-0.000015,0.000000,0.051605,0.108909,1.000
RATE_INTEREST_PRIMARY,5951.0,0.188357,0.087671,0.034781,0.160716,0.189122,0.193330,1.000
RATE_INTEREST_PRIVILEGED,5951.0,0.773503,0.100879,0.373150,0.715645,0.835095,0.852537,1.000
DAYS_DECISION,1670214.0,-880.679668,779.099667,-2922.000000,-1300.000000,-581.000000,-280.000000,-1.000


In [ ]:
dn_e = dn.groupby('SK_ID_CURR').median()
dn_e.fillna(dn_e.median(), inplace =True)

### Features Categoricas

In [ ]:
#Identificación de variables numéricas
det_app_C = det_app[(det_app['type']=='C')]
var_c = det_app_C['column'].tolist()
var_c.append('SK_ID_CURR')

dc = d[var_c]

In [ ]:
print('Visualización de las clases de las variables categóricas')
print('---------------------------------------------')
for c in var_c[:-1]:
    print ("%10s"%c, np.unique(dc[c].dropna()))

Visualización de las clases de las variables categóricas
---------------------------------------------
NAME_CONTRACT_TYPE ['Cash loans' 'Consumer loans' 'Revolving loans' 'XNA']
WEEKDAY_APPR_PROCESS_START ['FRIDAY' 'MONDAY' 'SATURDAY' 'SUNDAY' 'THURSDAY' 'TUESDAY' 'WEDNESDAY']
FLAG_LAST_APPL_PER_CONTRACT ['N' 'Y']
NFLAG_LAST_APPL_IN_DAY [0 1]
NAME_CASH_LOAN_PURPOSE ['Building a house or an annex' 'Business development' 'Buying a garage'
 'Buying a holiday home / land' 'Buying a home' 'Buying a new car'
 'Buying a used car' 'Car repairs' 'Education' 'Everyday expenses'
 'Furniture' 'Gasification / water supply' 'Hobby' 'Journey' 'Medicine'
 'Money for a third person' 'Other' 'Payments on other loans'
 'Purchase of electronic equipment' 'Refusal to name the goal' 'Repairs'
 'Urgent needs' 'Wedding / gift / holiday' 'XAP' 'XNA']
NAME_CONTRACT_STATUS ['Approved' 'Canceled' 'Refused' 'Unused offer']
NAME_PAYMENT_TYPE ['Cash through the bank' 'Cashless from the account of the employer'
 'Non

In [ ]:
dc.set_index('SK_ID_CURR', inplace=True)

In [ ]:
col_str = (dc.dtypes == object)
col_str = [c for c in col_str.index if col_str[c]]

col_du = (dc.dtypes != object)
col_du = [c for c in col_du.index if col_du[c]]

In [ ]:
dc_str = pd.get_dummies(dc[col_str], prefix=col_str)
dc_str.reset_index(inplace=True)
dc_str = dc_str.groupby('SK_ID_CURR').max().reset_index()
dc_str

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,100001,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,100002,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,100003,1,1,0,0,1,0,1,1,0,...,1,0,1,0,1,0,0,0,0,0
3,100004,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100005,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
338853,456252,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
338854,456253,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
338855,456254,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0


In [ ]:
dc_du = dc[col_du]
dc_du.fillna(0, inplace =True)
dc_du = dc_du.groupby('SK_ID_CURR').max().reset_index()
dc_du['NFLAG_INSURED_ON_APPROVAL'] = dc_du['NFLAG_INSURED_ON_APPROVAL'].astype('int64')
dc_du

,SK_ID_CURR,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL
0,100001,1,0
1,100002,1,0
2,100003,1,1
3,100004,1,0
4,100005,1,0
...,...,...,...
338852,456251,1,0
338853,456252,1,1
338854,456253,1,1
338855,456254,1,1


## Unión de datos Previous procesados

In [ ]:
# Validación de que las variables transformadas no contengan valores faltantes
dc_str.isnull().sum().sum(), dc_du.isnull().sum().sum(), dn_e.isnull().sum().sum()

(0, 0, 0)

In [ ]:
previous = pd.merge(dn_e,dc_str, how ='outer', on = 'SK_ID_CURR') 
previous = pd.merge(previous,dc_du, how ='outer', on = 'SK_ID_CURR') 
previous.isnull().sum().sum()

0

In [ ]:
previous.drop_duplicates(subset=['SK_ID_CURR'], inplace= True)

In [ ]:
col_pre = previous.columns.tolist()[1:]
for i in col_pre:
  previous.rename(columns={i:i+'_pre'}, inplace = True)

In [ ]:
previous.to_csv(ruta_lectura + "previous_procesada.csv")

# Aplication

In [ ]:
#Carga de tabla principal
d = pd.read_csv (ruta_lectura + 'application_train.csv')
#Se asigna el "ID" como index
d.set_index('SK_ID_CURR', inplace=True)
d.shape

(307511, 121)

In [ ]:
#Tipos de variables.
# archivo auxiliar construido para el entendimiento de las features
# ==============================================================================
# det_var = pd.read_excel('type_variables.xlsx')
det_var = pd.read_excel(ruta_lectura + 'type_variables.xlsx')
det_app = det_var[det_var['Table']=='application_{train|test}.csv']

# Se separa la data en distintos sets: 
#la variable objetivo y las Features de acuerdo a la necesidad de procesamiento

# set TARGET
# ==============================================================================
d_y = d['TARGET']

# set Features numericas
# ==============================================================================
det_app_N = det_app[(det_app['type']=='N')]
var_n = det_app_N['column'].tolist()
d_xn = d[var_n]

# set Features categoricas
# ==============================================================================
det_app_C = det_app[(det_app['type']=='C')]
var_c = det_app_C['column'].tolist()
d_xc = d[var_c]

In [ ]:
# Tratamiento de variables numéricas
# Imputación de valores faltantes con la mediana
d_xn.fillna(d_xn.median(), inplace =True)

In [ ]:
#Tratamiento de variables categóricas
col_str = (d_xc.dtypes == object)
col_str = [c for c in col_str.index if col_str[c]]

col_du = (d_xc.dtypes != object)
col_du = [c for c in col_du.index if col_du[c]]

In [ ]:
d_xc_str = pd.get_dummies(d_xc[col_str], prefix=col_str)

In [ ]:
d_xc_du = d_xc[col_du]

In [ ]:
# Validación de que las variables transformadas no contengan valores faltantes
d_xc_str.isnull().sum().sum(), d_xc_du.isnull().sum().sum(), d_xn.isnull().sum().sum()

(0, 0, 0)

# Generación del dataset ampliado

In [ ]:
# Unión de la data procesada
# Estructuración de la información para la implementación del modelo
d_x = pd.merge(d_xn, d_xc_du, how ='inner', left_index=True, right_index=True)
d_x = pd.merge(d_x, d_xc_str, how ='inner', left_index=True, right_index=True)
d_x.reset_index(inplace =True)

d_x = pd.merge(d_x, bureau, how = 'left', left_on= 'SK_ID_CURR', right_on='SK_ID_CURR')
d_x = pd.merge(d_x, previous, how = 'left', left_on= 'SK_ID_CURR', right_on='SK_ID_CURR')

# Imputación de valores faltantes con la mediana
d_x.fillna(d_x.median(), inplace =True)
d_x.isnull().sum().sum()
d_x.set_index('SK_ID_CURR', inplace=True)

In [ ]:
# Guardando el nombre de las features 
feature_name = d_x.columns.tolist()
pd.DataFrame(feature_name).to_csv(ruta_lectura + "feature_names_ampliada.csv", index = False)

In [ ]:
# Reescalamiento de datos
d_x_scaled = MinMaxScaler().fit_transform(d_x)

In [ ]:
# Función para evaluar la distribución de clases de una variable
def dis_clases(var):
  a1 = pd.DataFrame(pd.value_counts(var, normalize=True)).round(4).rename(columns = {var.name:'%'})
  a2 = pd.DataFrame(pd.value_counts(var)).rename(columns = {var.name:'Muestras'})
  a = pd.merge(a2, a1, how='inner', left_index=True, right_index=True)

  return a

In [ ]:
#División de la data en sets de Test y entrenamiento aplicando estratificación
# 20% Test y 80% Entrenamiento

# División estratificada
print('Distribución del target: Data train estratificada')

X_train, X_test, y_train, y_test = train_test_split(d_x_scaled, d_y, 
                                                    test_size=0.2, random_state=0, stratify=d_y)

print(dis_clases(y_train))
print('------------------------------------------------------------------------')
print(f'X_train: {X_train.shape}, X_test: {X_test.shape} ')
print(f'y_train: {y_train.shape}, y_test: {y_test.shape} ')

np.savetxt(ruta_lectura + "X_train_ampliada.csv", X_train, delimiter=",")
np.savetxt(ruta_lectura + "y_train_ampliada.csv", y_train, delimiter=",")

np.savetxt(ruta_lectura + "X_test_ampliada.csv", X_test, delimiter=",")
np.savetxt(ruta_lectura + "y_test_ampliada.csv", y_test, delimiter=",")

Distribución del target: Data train estratificada
   Muestras       %
0    226148  0.9193
1     19860  0.0807
------------------------------------------------------------------------
X_train: (246008, 441), X_test: (61503, 441) 
y_train: (246008,), y_test: (61503,) 


# Conclusión de la ejecución:

Se preprocesaron las dos tablas complementarias principales y se adhirieron a la data principal generando así datasets de entrenamiento y testeo ampliados para usarlos en posteriores iteraciones del modelo.